<a href="https://colab.research.google.com/github/hiimkhang/dog_breed_classification/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get dataset

In [29]:
import requests
import zipfile
import os

zip_file_path = "dog_breed_classification/small_dataset.zip"
if not os.path.isdir('small_dataset'):
    !git clone https://github.com/hiimkhang/dog_breed_classification
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall()


# Customize ResNet50 module

In [30]:
import torch
from torch import Tensor
import torch.nn as nn
from typing import Type, Any, Callable, Union, List, Optional
from torch.quantization import QuantStub, DeQuantStub


try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url


model_urls = {
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
}


def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion: int = 1

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride
        self.skip_add = nn.quantized.FloatFunctional()

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        #out += identity
        out = self.skip_add.add(out, identity)
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion: int = 4

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        self.skip_add = nn.quantized.FloatFunctional()

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        #out += identity
        out = self.skip_add.add(out, identity)
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
        num_classes: int = 1000,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation: Optional[List[bool]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        self.quant = QuantStub()
        self.dequant = DeQuantStub()
        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    def _make_layer(self, block: Type[Union[BasicBlock, Bottleneck]], planes: int, blocks: int,
                    stride: int = 1, dilate: bool = False) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.quant(x) # add quant
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        x = self.dequant(x) # add dequant

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)


def _resnet(
    arch: str,
    block: Type[Union[BasicBlock, Bottleneck]],
    layers: List[int],
    pretrained: bool,
    progress: bool,
    **kwargs: Any
) -> ResNet:
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model


def resnet50_quantizable(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-50 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet50', Bottleneck, [3, 4, 6, 3], pretrained, progress,
                   **kwargs)

# Prepare dataloader

In [31]:
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

from torch.utils.data import DataLoader

from torchvision import datasets
import torchvision.models as models
import torchvision.transforms as transforms
import os

root = './small_dataset'
img_size = 224

train_batch_size = 8
eval_batch_size = 8
num_workers = 2

loss_fn = torch.nn.CrossEntropyLoss()

train_transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                std=(0.229, 0.224, 0.225)),
    ])

test_transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                std=(0.229, 0.224, 0.225)),
    ])

train_set = datasets.ImageFolder(root=os.path.join(root, 'train'), transform=train_transform)
test_set = datasets.ImageFolder(root=os.path.join(root, 'valid'), transform=test_transform)

train_sampler = torch.utils.data.RandomSampler(train_set)
test_sampler = torch.utils.data.SequentialSampler(test_set)

train_loader = DataLoader(dataset=train_set,
                        batch_size=train_batch_size,
                        sampler=train_sampler,
                        num_workers=num_workers)

test_loader = DataLoader(dataset=test_set,
                        batch_size=eval_batch_size,
                        sampler=test_sampler,
                        num_workers=num_workers)

In [32]:
cuda_device = torch.device("cuda:0")
cpu_device = torch.device("cpu:0")

In [33]:
batch_inputs, batch_targets = next(iter(train_loader))

in_channels = batch_inputs.shape[1]     # number of channels of input images
out_features = len(train_set.classes)

print("Input tensor shape:", batch_inputs.shape)
print("Number of classes (out_features):", out_features)
print("in_channels: ", in_channels)

Input tensor shape: torch.Size([8, 3, 224, 224])
Number of classes (out_features): 3
in_channels:  3


# Helper training functions
- modifyLayers
- train_step
- eval_step
- train_model

In [34]:
def modifyLayers(model: torch.nn.Module, in_channels: int, out_features: int):
    # modify the first convolutional layer, in_channels = input tensor channels
    model.conv1 = torch.nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)

    # modify the last fc layer, out_features = number of classes
    model.fc = torch.nn.Linear(model.fc.in_features, out_features)

In [35]:
def test(model: torch.nn.Module, loaders: DataLoader):
    # Test the model
    model.to('cpu')
    model.eval()
    accuracy = 0
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in tqdm(loaders, total=len(loaders)):
            images = images.to('cpu')
            labels = labels.to('cpu')

            # Quantize the input data (images)
            # images_quantized = torch.quantization.quantize_data(images, scale=1.0, zero_point=0, dtype=torch.quint8)
            test_output = model(images)
            pred_y = torch.max(test_output, 1)[1].data.squeeze()
            accuracy = (pred_y == labels).sum().item() / float(labels.size(0))

    print(f"Test Accuracy of the model on the {len(loaders) * loaders.batch_size} test images: {accuracy:.2f}")


In [36]:
def train_step(model: nn.Module,
               train_loader: DataLoader,
               loss_fn: nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> tuple[float, float]:
    model.to(device)
    model.train()

    train_loss, train_accuracy = 0., 0.

    for batch, (image, label) in enumerate(train_loader):
        image, label = image.to(device), label.to(device)

        label_pred = model(image) # forward pass

        loss = loss_fn(label_pred, label) # compute loss
        train_loss += loss.item()

        optimizer.zero_grad() # zero param gradients

        loss.backward() # backward pass

        optimizer.step() # update parameters

        y_pred_labels = torch.argmax(torch.softmax(label_pred, dim=1), dim=1) # compute accuracy
        train_accuracy += (y_pred_labels == label).sum().item() / len(label_pred)

    avg_loss, avg_acc = train_loss/len(train_loader), train_accuracy/len(train_loader)
    print(f"Train loss: {avg_loss:.4f} | Train accuracy: {avg_acc:.4f}", end=' | ')

    return avg_loss, avg_acc

In [37]:
def eval_step(model: nn.Module,
              test_loader: DataLoader,
              loss_fn: nn.Module,
              device: torch.device) -> tuple[float, float]:
    model.to(device)
    model.eval()

    test_loss, test_accuracy = 0., 0.

    with torch.inference_mode():
        for batch, (image, label) in enumerate(test_loader):
            image, label = image.to(device), label.to(device)

            label_pred_logits = model(image) # forward pass

            loss = loss_fn(label_pred_logits, label) # compute loss
            test_loss += loss.item()

            label_pred_labels = label_pred_logits.argmax(dim=1) # compute accuracy
            test_accuracy += (label_pred_labels == label).sum().item() / len(label_pred_labels)

    avg_loss, avg_acc = test_loss/len(test_loader), test_accuracy/len(test_loader)
    print(f"Test loss: {avg_loss:.4f} | Test accuracy: {avg_acc:.4f}")

    return avg_loss, avg_acc

In [38]:
def train_model(model: nn.Module,
          train_loader: DataLoader,
          test_loader: DataLoader,
          epochs: int,
          device: torch.device) -> dict[str, list]:
    global results_info
    model.to(device)

    results = {
        'Train loss': [],
        'Train accuracy': [],
        'Test loss': [],
        'Test accuracy': [],
    }

    optimizer = optim.SGD(model.parameters(),
                            1e-3,
                            momentum=0.9,
                            weight_decay=1e-4)

    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,
                                                     milestones=[100, 150],
                                                     gamma=0.1,
                                                     last_epoch=-1)

    for epoch in range(epochs):

        print(f"\nEpoch {epoch+1}/{epochs}: ", end="")

        train_loss, train_acc = train_step(model, train_loader, loss_fn, optimizer, device)
        test_loss, test_acc = eval_step(model, test_loader, loss_fn, device)

        results['Train loss'].append(train_loss)
        results['Train accuracy'].append(train_acc)
        results['Test loss'].append(test_loss)
        results['Test accuracy'].append(test_acc)

        scheduler.step()
        # Write model into log.txt
        # with open('log.txt', 'w') as f:
        #     f.write(f"Model: {model}")

    return results

# Model loading and training

In [39]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

# Use an original resnet50 pretrained model to train and evaluate
# Can be used for later comparisons
resnet50 = models.resnet50(pretrained=True)

modifyLayers(model=resnet50,
             in_channels=in_channels,
             out_features=out_features) # modify first conv1 and last fc layer according to dataset

resnet50_result = train_model(model=resnet50,
            train_loader=train_loader,
            test_loader=test_loader,
            epochs=3,
            device=cuda_device)


Epoch 1/3: Train loss: 1.0761 | Train accuracy: 0.3995 | Test loss: 1.1252 | Test accuracy: 0.3854

Epoch 2/3: Train loss: 0.9122 | Train accuracy: 0.5543 | Test loss: 0.8148 | Test accuracy: 0.5583

Epoch 3/3: Train loss: 0.7314 | Train accuracy: 0.6467 | Test loss: 0.9088 | Test accuracy: 0.5812


In [40]:
# Use customized resnet50 pretrained model to train and evaluate
# This quantizable model later will be used to quantize into the compressed int8 param model

resnet50_quantizable = resnet50_quantizable(pretrained=True)

modifyLayers(model=resnet50_quantizable,
             in_channels=in_channels,
             out_features=out_features) # modify first conv1 and last fc layer according to dataset

resnet50_quantizable_result = train_model(model=resnet50_quantizable,
            train_loader=train_loader,
            test_loader=test_loader,
            epochs=3,
            device=cuda_device)


Epoch 1/3: Train loss: 1.0457 | Train accuracy: 0.4484 | Test loss: 0.8763 | Test accuracy: 0.5021

Epoch 2/3: Train loss: 0.8534 | Train accuracy: 0.5924 | Test loss: 0.7063 | Test accuracy: 0.6542

Epoch 3/3: Train loss: 0.6069 | Train accuracy: 0.7092 | Test loss: 0.8037 | Test accuracy: 0.6292


In [41]:
# Use the original resnet50 and customized quantizable resnet50 to evaluate on the test
print("ResNet50: ", end="")
resnet50_eval = eval_step(resnet50, test_loader, loss_fn, cpu_device)

print("Quantizable ResNet50: ", end="")
resnet50_quantizable_eval = eval_step(resnet50_quantizable, test_loader, loss_fn, cpu_device)

ResNet50: Test loss: 0.9088 | Test accuracy: 0.5812
Quantizable ResNet50: Test loss: 0.8037 | Test accuracy: 0.6292


In [42]:
# Save the two trained models
if not os.path.isdir('save_models'):
    os.makedirs('save_models', exist_ok=True)

torch.save(resnet50.state_dict(), "./save_models/resnet50_trained.pt")
torch.save(resnet50_quantizable.state_dict(), "./save_models/resnet50_quantizable_trained.pt")

# Post-training static quantization

In [55]:
def calibrate_model(model: nn.Module, loader: DataLoader, device=torch.device('cpu:0')):
  model.to(device)
  model.eval()

  for image, label in tqdm(loader, total=len(loader)):
    image, label = image.to(device), label.to(device)
    model(image)

In [62]:
import copy

# Our initial baseline model which is FP32
model_fp32 = copy.deepcopy(resnet50_quantizable)

# Move the model to CPU since static quantization does not support CUDA currently.
model_fp32.to(cpu_device)

model_fp32.eval()   # set to evaluation mode, a requirement for below layers fusing

# Fuse layers
model_fp32 = torch.quantization.fuse_modules(model_fp32, [['conv1', 'bn1', 'relu']], inplace=True)

# Set the backend for x86
model_fp32.qconfig = torch.quantization.get_default_qconfig('fbgemm')

# Prepares the model for calibration.
# Inserts observers in the model that will observe the activation tensors during calibration
model_fp32_prepared = torch.quantization.prepare(model_fp32, inplace = False)

# Calibrate over the train dataset. This determines the quantization params for activation.
# Use training data for calibration.
calibrate_model(model=model_fp32_prepared,
                loader=train_loader,
                device=cpu_device)

# for batch, target in tqdm(train_loader, total=len(train_loader)):
#     model_fp32_prepared(batch)

# Converts the model to a quantized model (int8)
resnet50_quantized = torch.quantization.convert(model_fp32_prepared)

100%|██████████| 12/12 [00:32<00:00,  2.75s/it]


In [65]:
if not os.path.isdir('save_models'):
    os.makedirs('save_models', exist_ok=True)

torch.save(resnet50_quantized.state_dict(), "./save_models/resnet50_quantized.pt")

In [63]:
# Use the compressed quantized model to evaluate the test set
resnet50_quantized_eval = eval_step(resnet50_quantized, test_loader, loss_fn, cpu_device)

Test loss: 0.8316 | Test accuracy: 0.6250


(0.8316468851019939, 0.625)

In [71]:
# Compare to the previous Quantizable ResNet50 and original ResNet50:
print(f"ResNet50 accuracy: {resnet50_eval[1]:.4f}")
print(f"Quantizable ResNet50 accuracy: {resnet50_quantizable_eval[1]:.4f}")

ResNet50 accuracy: 0.5812
Quantizable ResNet50 accuracy: 0.6292


# **Conclusion**